In [1]:
# Loading ColorChecker24 Lab_D50 value from X-rite data(after 2015)
import numpy as np

def parse_data_file(file_path):
    # Deepseek
    # 初始化存储变量
    string_list = []
    numeric_data = []
    
    # 标志是否开始/结束读取数据
    reading_data = False
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # 去除首尾空白字符
            
            if not reading_data:
                if line == 'BEGIN_DATA':
                    reading_data = True
                continue
            else:
                if line == 'END_DATA':
                    break
                
                # 分割数据行
                parts = line.split('\t')
                if len(parts) != 4:
                    continue  # 跳过格式不正确的行
                
                # 解析数据
                string_part = parts[0]
                try:
                    numeric_parts = [float(x) for x in parts[1:4]]
                except ValueError:
                    continue  # 跳过数值转换失败的行
                
                # 存储数据
                string_list.append(string_part)
                numeric_data.append(numeric_parts)
    
    # 将数值数据转换为N×3的NumPy数组
    numeric_array = np.array(numeric_data)
    
    return string_list, numeric_array

patch_name, lab_values = parse_data_file('RefPatches.txt')
# The value is column first, where A* is the first column.
patch_name_row_first = np.array(patch_name).reshape(4, 6, order='F').ravel(order='C')
lab_values_row_first = lab_values.reshape((4, 6, 3), order='F').reshape((-1, 3), order='C')
print(patch_name_row_first)
print(lab_values_row_first)


['A1' 'B1' 'C1' 'D1' 'E1' 'F1' 'A2' 'B2' 'C2' 'D2' 'E2' 'F2' 'A3' 'B3'
 'C3' 'D3' 'E3' 'F3' 'A4' 'B4' 'C4' 'D4' 'E4' 'F4']
[[ 3.754e+01  1.437e+01  1.492e+01]
 [ 6.466e+01  1.927e+01  1.750e+01]
 [ 4.932e+01 -3.820e+00 -2.254e+01]
 [ 4.346e+01 -1.274e+01  2.272e+01]
 [ 5.494e+01  9.610e+00 -2.479e+01]
 [ 7.048e+01 -3.226e+01 -3.700e-01]
 [ 6.273e+01  3.583e+01  5.650e+01]
 [ 3.943e+01  1.075e+01 -4.517e+01]
 [ 5.057e+01  4.864e+01  1.667e+01]
 [ 3.010e+01  2.254e+01 -2.087e+01]
 [ 7.177e+01 -2.413e+01  5.819e+01]
 [ 7.151e+01  1.824e+01  6.737e+01]
 [ 2.837e+01  1.542e+01 -4.980e+01]
 [ 5.438e+01 -3.972e+01  3.227e+01]
 [ 4.243e+01  5.105e+01  2.862e+01]
 [ 8.180e+01  2.670e+00  8.041e+01]
 [ 5.063e+01  5.128e+01 -1.412e+01]
 [ 4.957e+01 -2.971e+01 -2.832e+01]
 [ 9.519e+01 -1.030e+00  2.930e+00]
 [ 8.129e+01 -5.700e-01  4.400e-01]
 [ 6.689e+01 -7.500e-01 -6.000e-02]
 [ 5.076e+01 -1.300e-01  1.400e-01]
 [ 3.563e+01 -4.600e-01 -4.800e-01]
 [ 2.064e+01  7.000e-02 -4.600e-01]]


In [7]:
%matplotlib qt
# Generate reference ColorChecker24 sRGB chart image
import numpy as np
import matplotlib.pyplot as plt
import colour

# 1. ColorChecker 24 色块在 D50 光源下的 L*a*b* 值
import numpy as np

def create_colorchecker_data_from_matrix(lab_matrix, patch_names):
    """
    将一个 24x3 的 L*a*b* NumPy 矩阵转换为 ColorChecker 字典数据结构。

    参数:
    ----------
    lab_matrix : numpy.ndarray
        一个形状为 (24, 3) 的 NumPy 数组，每一行代表一个色块的 L*a*b* 值。
    patch_names : list
        一个包含 24 个字符串的列表，按顺序对应矩阵中的每一行色块。

    返回:
    -------
    dict
        一个字典，键是色块名称，值是包含 'Lab' 键和 L*a*b* 数组的另一个字典。
        例如: {'dark skin': {'Lab': array([37.15, 13.25, 14.24])}, ...}
    """
    # 输入验证
    if not isinstance(lab_matrix, np.ndarray) or lab_matrix.shape != (24, 3):
        raise ValueError("输入 'lab_matrix' 必须是一个 shape 为 (24, 3) 的 NumPy 数组。")
    if not isinstance(patch_names, list) or len(patch_names) != 24:
        raise ValueError("输入 'patch_names' 必须是一个包含 24 个名称的列表。")

    # 创建字典
    color_data_dict = {}
    for i, name in enumerate(patch_names):
        # 从矩阵中提取对应行的 L*a*b* 值
        lab_value = lab_matrix[i]
        # 构建所需的嵌套字典结构
        color_data_dict[name] = {"Lab": lab_value}
        
    return color_data_dict

# 按标准 4x6 顺序排列的色块名称
PATCH_ORDER = [
    "dark skin", "light skin", "blue sky", "foliage", "blue flower", "bluish green",
    "orange", "purplish blue", "moderate red", "purple", "yellow green", "orange yellow",
    "blue", "green", "red", "yellow", "magenta", "cyan",
    "white", "neutral 8", "neutral 6.5", "neutral 5", "neutral 3.5", "black"
]

# 1. 从 ColorChecker 24 数据中提取 L*a*b* 值
COLORCHECKER_DATA = create_colorchecker_data_from_matrix(lab_values_row_first, PATCH_ORDER)


# 2. 定义源光源和目标光源
ILLUMINANT_D50 = colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D50']
ILLUMINANT_D65 = colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D65']
color_correction_mtx_cat02 = colour.adaptation.matrix_chromatic_adaptation_VonKries(
    XYZ_w=colour.xyY_to_XYZ(colour.xy_to_xyY(colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D50'])),
    XYZ_wr=colour.xyY_to_XYZ(colour.xy_to_xyY(colour.CCS_ILLUMINANTS['CIE 1931 2 Degree Standard Observer']['D65'])),
    transform='Bradford',
)

# 3. 进行颜色转换
sRGB_colors = {}
for name in PATCH_ORDER:
    lab_d50 = COLORCHECKER_DATA[name]["Lab"]
    
    # Lab (D50) -> XYZ (D50)
    xyz_d50 = colour.Lab_to_XYZ(lab_d50, illuminant=ILLUMINANT_D50)
    
    # XYZ (D50) -> XYZ (D65) (使用 CAT02 色度适应方法)
    xyz_d65 = xyz_d50 @ color_correction_mtx_cat02.T
    
    # XYZ (D65) -> sRGB (D65)
    srgb = colour.XYZ_to_sRGB(xyz_d65)
    
    # 4. 将 sRGB 值限制在 [0, 1] 范围内，以防超出色域
    srgb_clipped = np.clip(srgb, 0, 1)
    
    sRGB_colors[name] = srgb_clipped

# 将原始顺序的列表转换为 4x6 的网格
original_grid = np.array(PATCH_ORDER).reshape(4, 6)
# 将网格逆时针旋转90度 (k=1 表示逆时针旋转1次)
rotated_grid = np.rot90(original_grid, k=1)
# 将旋转后的 6x4 网格展平，得到新的填充顺序
ROTATED_PATCH_ORDER = rotated_grid.flatten().tolist()


# 6. 使用 Matplotlib 可视化 (代码与之前相同)
fig, axes = plt.subplots(4, 6, figsize=(12, 8))
fig.suptitle('My Custom ColorChecker (sRGB from Lab D50)', fontsize=16)
axes = axes.ravel()

for i, name in enumerate(PATCH_ORDER):
    ax = axes[i]
    color = sRGB_colors[name]
    ax.set_facecolor(color)
    ax.set_xticks([])
    ax.set_yticks([])
    luminance = COLORCHECKER_DATA[name]["Lab"][0]
    text_color = 'white' if luminance < 50 else 'black'
    # ax.text(0.5, 0.5, name, color=text_color, ha='center', va='center', fontsize=9, wrap=True)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()